# Demo: Externally Defined Layouts
This notebook computes a layout using NetworkX and sends it to Graphistry for subsequent interactive analysis.

In [1]:
import graphistry
#graphistry.register(key='MY_KEY', server='labs.graphistry.com') #https://www.graphistry.com/api-request
import networkx as nx
import pandas as pd

## Make a NetworkX Graph

In [2]:
num_nodes = 1000
G=nx.Graph()
G.add_nodes_from(range(num_nodes))
Edges = [(x, (x * 2) % (num_nodes/5)) for x in range(num_nodes)] + [(x, (x + 1) % 20) for x in range(20)]
G.add_edges_from(Edges)
G

## Use a NetworkX layout

In [3]:
%time
pos=nx.fruchterman_reingold_layout(G)
pos[0]

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


array([0.3766924 , 0.36150154], dtype=float32)

## Combine into node, edge dataframes

In [4]:
def pos2df (pos):
    nodes = pd.DataFrame({'key': pos.keys(), 'pos_0': [pos[k][0] for k in pos.keys()], 'pos_1': [pos[k][1] for k in pos.keys()]})
    nodes.columns = ['key', 'x', 'y']
    return nodes


In [5]:
nodes = pos2df(pos)
nodes[:3]

,key,x,y
0,0,0.376692,0.361502
1,1,0.483908,0.427637
2,2,0.588375,0.491546


In [6]:
edges = pd.DataFrame(Edges)
edges.columns = ['src', 'dst']
edges[:3]

,src,dst
0,0,0
1,1,2
2,2,4


## Autolayout mode

In [7]:
g = graphistry.nodes(nodes).edges(edges).bind(source='src', destination='dst', node='key')
g.plot()

## Predefined layout mode
As the node table provides "x" and "y" columns, they will be automatically used as starting positions. To prevent the automatic layout algorithm from moving the nodes on load, we also set the URL parameter "play" to 0 seconds. (Both settings will likely change.)

In [8]:
g2 = g.settings(url_params={'play':0})
g.plot()

## For fun, here's a circular layout

In [9]:
pos2=nx.circular_layout(G, scale=100)
nodes2 = pos2df(pos2)
g2.nodes(nodes2).plot()